In [1]:
!pip install keras-tuner

     |████████████████████████████████| 71kB 5.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=dd6682dd74bead5834d929822cfcded5556981a48b6d9692a7af314bd7a2b3ad
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=b8a3ece1baf2464fc4a02f01116977a6bae22a9d6a7c3df12f4398727e8813d8
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [3]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [4]:
fashion_mnist=keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [10]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values=[3,5]),
        activation='relu',
        input_shape=(28,28,1)
  ),
  keras.layers.Conv2D(
      filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
      kernel_size=hp.Choice('conv_2_kernel', values=[3,5]),
      activation='relu'
  ),
  keras.layers.Flatten(),
  keras.layers.Dense(    
      units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
      activation='relu',
  ),
  keras.layers.Dense(10, activation='softmax')    
  ]) 

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [11]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [12]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist_Fashion")

In [ ]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)


Search: Running Trial #12

Hyperparameter    |Value             |Best Value So Far 
conv_1_filter     |64                |48                
conv_1_kernel     |5                 |3                 
conv_2_filter     |48                |32                
conv_2_kernel     |5                 |5                 
dense_1_units     |128               |32                
learning_rate     |0.01              |0.001             

Epoch 1/3
1688/1688 [==============================] - 341s 202ms/step - loss: 0.5129 - accuracy: 0.8228 - val_loss: 0.4214 - val_accuracy: 0.8432
Epoch 2/3
1688/1688 [==============================] - 342s 203ms/step - loss: 0.3788 - accuracy: 0.8592 - val_loss: 0.5033 - val_accuracy: 0.8278
Epoch 3/3
1591/1688 [===========================>..] - ETA: 18s - loss: 0.3504 - accuracy: 0.8705

In [22]:
model=tuner_search.get_best_models(num_models=1)[0]

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 48)        480       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        38432     
_________________________________________________________________
flatten (Flatten)            (None, 15488)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                495648    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
Total params: 534,890
Trainable params: 534,890
Non-trainable params: 0
_________________________________________________________________
